In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
import pyperclip
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import requests as req
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
  # 웹드라이버 실행
driver = webdriver.Chrome()

# 제품 목록 페이지 로드
driver.get("https://www.flipkart.com/search?sid=j9e%2Fabm%2F8qx&otracker=CLP_Filters&p%5B%5D=facets.brand%255B%255D%3DSAMSUNG")

# 페이지 로딩 대기
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'CGtC98')))

# URL을 저장할 리스트 초기화
url_list = []

# 요소 찾기
elements = driver.find_elements(By.CLASS_NAME, 'CGtC98')

# 각 요소에서 'href' 속성을 가져와 리스트에 추가 (sponsored 요소는 제외)
for element in elements:
    try:
        # 'f8qK5m' 클래스를 가진 자식 요소가 있는지 확인
        if element.find_elements(By.CLASS_NAME, 'f8qK5m'):
            # 'f8qK5m' 클래스를 가진 자식 요소가 있다면 skip
            continue
        # 자식 요소가 'f8qK5m' 클래스를 가지고 있지 않다면 href 추출
        url_value = element.get_attribute('href')
        if url_value:  # href 속성이 있는 경우만 추가
            url_list.append(url_value)
    except NoSuchElementException:
        # 자식 요소가 없으면 그냥 넘어가기
        continue 
        
# 리뷰 데이터를 저장할 리스트 초기화
all_reviews = []

for url in tqdm(url_list):
    try:
        driver.get(url)
        
        # XPath를 사용하여 제목 찾기. 제품 코드 별 XPATH 달라서 접근 불가... 제목에서 코드 추출하세용...
        try:
            # XPath를 사용하여 제목 찾기. 제품 코드 별 XPATH 달라서 접근 불가... 제목에서 코드 추출하세용... 
            list_items = driver.find_elements(By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[2]/div/div[1]/h1/span')
            
            # 텍스트 추출
            hp_data = [item.text for item in list_items]
            if not hp_data:
                hp_data = ["No data"]  # 데이터가 없을 경우 기본값
        except (NoSuchElementException, TimeoutException) as e:
            hp_data = ["Error fetching data"]  # 오류 발생 시 기본값
        
        # 리뷰 버튼 클릭 대기
        try:
            review_button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "_23J90q.RcXBOT")))
            if review_button.is_displayed():
                review_button.click()
        except (NoSuchElementException, TimeoutException):
            pass
        
        # 낮은 별점 접근
        try:
            low_rating_option = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#container > div > div.VLIitu > div > div > div.DOjaWF.gdgoEp.col-9-12 > div.DOjaWF.gdgoEp.col-12-12 > div > div:nth-child(2) > div > div > div > div > select > option:nth-child(4)")))
            low_rating_option.click()
        except (NoSuchElementException, TimeoutException) as e:
            print(f"Error selecting sort option for URL {url}: {e}")
        
        # 데이터 확보
        page_number = 1
        
        while True:
            try:
                # 페이지 URL 업데이트
                current_url = driver.current_url
                full_url = f'{current_url}&page={page_number}'
                driver.get(full_url)
                
                # ZmyHeo 클래스의 존재 여부를 확인
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ZmyHeo')))
                
                # 데이터 추출
                text_elements = driver.find_elements(By.CLASS_NAME, 'ZmyHeo')
                state_elements = driver.find_elements(By.CLASS_NAME, 'MztJPv')
                date_elements = driver.find_elements(By.CLASS_NAME, '_2NsDsF')
                star_elements = driver.find_elements(By.XPATH, "//*[contains(@class, 'XQDdHH') and (contains(@class, 'Js30Fc') or contains(@class, 'Czs3gR') or contains(@class, 'Ga3i8K'))]")
                
                text_list = [elem.text for elem in text_elements]
                state_list = [elem.text for elem in state_elements]
                date_list = [elem.text for elem in date_elements]
                dates = date_list[1::2]
                star_list = [elem.text for elem in star_elements]
                
                # 리뷰 데이터를 모든 리뷰 리스트에 추가
                for t, s, d, r in zip(text_list, state_list, dates, star_list):
                    for data in hp_data:
                        all_reviews.append({
                            'text': t,
                            'state': s,
                            'date': d,
                            'star': r,
                            'hp_data': data  # 각각의 hp_data 항목을 추가
                        })

                # 페이지 번호 증가
                page_number += 1
                
            except (NoSuchElementException, TimeoutException) as e:
                print(f"Error on page {page_number} for URL {full_url}: {e}")
                break
    
    except Exception as e:
        print(f"Error processing URL {url}: {e}")

# 현재 창 닫기
driver.quit()

  8%|██████▍                                                                            | 1/13 [00:30<06:06, 30.58s/it]

Error processing URL https://www.flipkart.com/samsung-8-kg-fully-automatic-front-load-washing-machine-in-built-heater-black-grey/p/itm172f0f21c5c72?pid=WMNGWY2GVJKTGFMA&lid=LSTWMNGWY2GVJKTGFMAHYBDRP&marketplace=FLIPKART&store=j9e%2Fabm%2F8qx&srno=b_1_4&otracker=CLP_Filters&iid=cbeaec52-38a5-4e46-aa66-eb90302e95a6.WMNGWY2GVJKTGFMA.SEARCH&ssid=0orj4zosao0000001724716339325: name 'NoSuchElementException' is not defined


 15%|████████████▊                                                                      | 2/13 [01:01<05:41, 31.04s/it]

Error processing URL https://www.flipkart.com/samsung-7-kg-5-star-ai-wi-fi-digital-inverter-fully-automatic-front-load-washing-machine-in-built-heater-silver/p/itmb2a2336207b22?pid=WMNG87FQVRVSZ5DF&lid=LSTWMNG87FQVRVSZ5DFEQEBP1&marketplace=FLIPKART&store=j9e%2Fabm%2F8qx&srno=b_1_5&otracker=CLP_Filters&iid=cbeaec52-38a5-4e46-aa66-eb90302e95a6.WMNG87FQVRVSZ5DF.SEARCH&ssid=0orj4zosao0000001724716339325: name 'NoSuchElementException' is not defined


 23%|███████████████████▏                                                               | 3/13 [01:30<05:00, 30.02s/it]

...

Error processing URL https://www.flipkart.com/samsung-7-5-kg-5-star-air-turbo-drying-semi-automatic-top-load-washing-machine-blue-grey/p/itmecde21cc327fa?pid=WMNGGSK86PGTMGHX&lid=LSTWMNGGSK86PGTMGHXHX6UQB&marketplace=FLIPKART&store=j9e%2Fabm%2F8qx&srno=b_1_20&otracker=CLP_Filters&iid=cbeaec52-38a5-4e46-aa66-eb90302e95a6.WMNGGSK86PGTMGHX.SEARCH&ssid=0orj4zosao0000001724716339325: name 'NoSuchElementException' is not defined


 92%|███████████████████████████████████████████████████████████████████████████▋      | 12/13 [06:40<00:33, 33.54s/it]

Error processing URL https://www.flipkart.com/samsung-7-kg-5-star-ecobubble-wi-fi-digital-inverter-hygiene-steam-fully-automatic-top-load-washing-machine-in-built-heater-grey/p/itm0484e1b609cf4?pid=WMNGGUWZU2HZGVQX&lid=LSTWMNGGUWZU2HZGVQXKINO8Z&marketplace=FLIPKART&store=j9e%2Fabm%2F8qx&srno=b_1_23&otracker=CLP_Filters&iid=cbeaec52-38a5-4e46-aa66-eb90302e95a6.WMNGGUWZU2HZGVQX.SEARCH&ssid=0orj4zosao0000001724716339325: name 'NoSuchElementException' is not defined


100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [07:17<00:00, 33.67s/it]

Error processing URL https://www.flipkart.com/samsung-7-kg-5-star-hygiene-steam-inbuilt-heater-led-display-diamond-drum-digital-inverter-fully-automatic-front-load-washing-machine-silver/p/itm4452b8a8e27e9?pid=WMNGVGCN4EQPKUPE&lid=LSTWMNGVGCN4EQPKUPEEK3YEW&marketplace=FLIPKART&store=j9e%2Fabm%2F8qx&srno=b_1_24&otracker=CLP_Filters&iid=cbeaec52-38a5-4e46-aa66-eb90302e95a6.WMNGVGCN4EQPKUPE.SEARCH&ssid=0orj4zosao0000001724716339325: name 'NoSuchElementException' is not defined


In [3]:
df_SS_wash = pd.DataFrame(all_reviews)
df_SS_wash

,text,state,date,star,hp_data
0,Product is making loud rattling noise,"Certified Buyer, Pune",9 months ago,1,SAMSUNG 8 kg Fully Automatic Front Load Washin...
1,Very bad experience with installation.\nProduc...,"Certified Buyer, Vashind","Oct, 2022",1,SAMSUNG 8 kg Fully Automatic Front Load Washin...
2,The touch pad of the machine has issues from d...,"Certified Buyer, Mumbai",2 days ago,1,SAMSUNG 8 kg Fully Automatic Front Load Washin...
3,Not satisfied,"Certified Buyer, Vijayawada",10 days ago,1,SAMSUNG 8 kg Fully Automatic Front Load Washin...
4,After 2 days using having issue of water suppl...,"Certified Buyer, Hyderabad",13 days ago,1,SAMSUNG 8 kg Fully Automatic Front Load Washin...
...,...,...,...,...,...
1397,This machine does not work on normal tap water...,"Certified Buyer, Vasai Virar","Apr, 2022",1,"SAMSUNG 7 kg 5 star, Hygiene Steam with Inbuil..."
1398,the product was delivered on 24 Mar 22. Very v...,"Certified Buyer, Bharatpur District","Mar, 2022",1,"SAMSUNG 7 kg 5 star, Hygiene Steam with Inbuil..."
1399,Machine acts insane if a single extra cloth ad...,"Certified Buyer, Navi Mumbai","Mar, 2022",1,"SAMSUNG 7 kg 5 star, Hygiene Steam with Inbuil..."
1400,Drum is shaking in 2 years.,"Certified Buyer, Pune","Feb, 2022",1,"SAMSUNG 7 kg 5 star, Hygiene Steam with Inbuil..."


In [4]:
df_SS_wash.to_csv("df_SS_wash.csv")